In [85]:
import pandas as pd
import numpy as np
import requests
import datetime

### Extracción de data con la API de `CryptoCompare`

#### `NOTA:` Verificamos APIs de diferentes proveedores y no se encontró información de historial de `suministro de monedas`, si existe la data pero es necesario una API de pago, en este caso al ser un proyecto independiente y en preproducción optamos por usar `CriptoCompare` el cual una de sus APIs deja extraer esta data en forma de historial pero solo se encuentra disponible Bitcoin y Ethereum.

In [59]:
# Creamos una lista a diferencia de Coingecko lo hacemos con el símbolo, solo hay data de BTC y ETH 
crypto_list = ['BTC', 'ETH']

In [60]:
# Función para extraer datos con la API, se incluye sistema para detectar errores
def extract_data_blockchain(crypto):
    try:
        response = requests.get(
            'https://min-api.cryptocompare.com/data/blockchain/histo/day',
            params={"fsym": crypto, "apiKey": "9ea7da5f495d73e9a371c56ae3990d91929bba7001cc0db1294d174bcb75781b", "limit": "2000", "toTs": "1519794000"}, # "toTs": "1519794000" se configuró para extraer el segundo tramo de btc y eth
            headers={"Content-type": "application/json; charset=UTF-8"}
        )
        response.raise_for_status()  # Lanza una excepción si la respuesta no es exitosa (por ejemplo, código de estado HTTP 404)

        data = response.json()
        
        # Imprimir la respuesta completa para verificar su estructura
        print(data)

        if 'Data' in data:
            df = pd.json_normalize(data['Data']['Data'])
            # Convierte el sistema de hora Unix a YYYY-MM-DD
            df["time"] = [datetime.datetime.utcfromtimestamp(x).strftime("%Y-%m-%d") for x in df["time"]]
            return df
        else:
            # En caso de que no haya datos en la respuesta
            print(f"No se encontraron datos para {crypto}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error de solicitud: {e}")
        return None

In [53]:
#creamos un diccionario para almacenar los dataframe
df_blocks = {}
# Bucle para extraer los datos de la lista de criptomonedas creadas anteriormente
for cryptocurrency in crypto_list:
    df = extract_data_blockchain(cryptocurrency)
    if df is not None:
        # Guardamos cada dataframe en el diccionario
        df_blocks[cryptocurrency] = df
    else:
        print(f"No se pudo obtener datos para {cryptocurrency}")

{'Response': 'Success', 'Message': '', 'HasWarning': False, 'Type': 100, 'RateLimit': {}, 'Data': {'Aggregated': False, 'TimeFrom': 1519776000, 'TimeTo': 1692576000, 'Data': [{'id': 1182, 'symbol': 'BTC', 'time': 1519776000, 'zero_balance_addresses_all_time': 354073511, 'unique_addresses_all_time': 378480916, 'new_addresses': 275526, 'active_addresses': 809779, 'transaction_count': 189223, 'transaction_count_all_time': 302072984, 'large_transaction_count': 11966, 'average_transaction_value': 7.30930158085989, 'block_height': 511384, 'hashrate': 23194211.4053753, 'difficulty': 3007383866429.73, 'block_time': 556.889610389611, 'block_size': 1054216, 'current_supply': 16892289.7952987}, {'id': 1182, 'symbol': 'BTC', 'time': 1519862400, 'zero_balance_addresses_all_time': 354486975, 'unique_addresses_all_time': 378828064, 'new_addresses': 347148, 'active_addresses': 824240, 'transaction_count': 233744, 'transaction_count_all_time': 302306728, 'large_transaction_count': 12767, 'average_trans

In [61]:
# Este fragmento hace la misma función que el anterior, la diferencia es que la API tiene una limitación de 2000 
# registros y esta segunda extrae el restante necesario para nivelar nuestra data con la de coingecko
# creamos un diccionario para almacenar los dataframe
df_blocks_2 = {}
for cryptocurrency in crypto_list:
    df_2 = extract_data_blockchain(cryptocurrency)
    if df_2 is not None:
        # Guardamos cada dataframe en el diccionario
        df_blocks_2[cryptocurrency] = df_2
    else:
        print(f"No se pudo obtener datos para {cryptocurrency}")

{'Response': 'Success', 'Message': '', 'HasWarning': False, 'Type': 100, 'RateLimit': {}, 'Data': {'Aggregated': False, 'TimeFrom': 1346976000, 'TimeTo': 1519776000, 'Data': [{'id': 1182, 'symbol': 'BTC', 'time': 1346976000, 'zero_balance_addresses_all_time': 5437955, 'unique_addresses_all_time': 6277056, 'new_addresses': 24191, 'active_addresses': 44106, 'transaction_count': 33483, 'transaction_count_all_time': 6871034, 'large_transaction_count': 576, 'average_transaction_value': 549.885765315252, 'block_height': 197769, 'hashrate': 22.479166031691, 'difficulty': 2694047.95295501, 'block_time': 514.736526946108, 'block_size': 83443, 'current_supply': 9888490.00531868}, {'id': 1182, 'symbol': 'BTC', 'time': 1347062400, 'zero_balance_addresses_all_time': 5455455, 'unique_addresses_all_time': 6296502, 'new_addresses': 19446, 'active_addresses': 33700, 'transaction_count': 31173, 'transaction_count_all_time': 6902207, 'large_transaction_count': 404, 'average_transaction_value': 243.296513

In [76]:
# Declaramos las variables concatenando las dos extracciones, reune los rangos de fecha totales
df_supply_bitcoin = pd.concat([df_blocks_2["BTC"], df_blocks["BTC"]]).copy()
df_supply_ethereum = pd.concat([df_blocks_2["ETH"], df_blocks["ETH"]]).copy()

#### Función para realizar limpieza, eliminar columnas poco relevantes

In [82]:
def clean_data(df_crypto):
    df_crypto = df_crypto.astype({'id': 'string'})
    df_crypto['id'] = df_crypto['id'].str.replace("1182", "bitcoin", case=False, regex=False)
    df_crypto['id'] = df_crypto['id'].str.replace("7605", "ethereum", case=False, regex=False)
    df_crypto = df_crypto.drop(columns=['zero_balance_addresses_all_time', 'unique_addresses_all_time',	'new_addresses', 'active_addresses', 'transaction_count', 'transaction_count_all_time', 'large_transaction_count', 'average_transaction_value',	'block_height',	'hashrate',	'difficulty', 'block_time', 'block_size'])
    df_crypto = df_crypto.dropna(subset=['current_supply'])
    df_crypto = df_crypto.astype({'current_supply': 'int64'})
    df_crypto = df_crypto.drop(columns=['symbol'])
    df_crypto.rename(columns={'id': 'cryptocurrency_id'}, inplace=True)
    df_crypto = df_crypto.astype({'time': 'datetime64[ns]'})
    df_crypto = df_crypto.rename(columns={'time': 'timestamp'})
    return df_crypto

In [73]:
# Activamos la función y guardamos sobre la misma varible
df_supply_bitcoin = clean_data(df_supply_bitcoin)

In [75]:
# Guardamos en un dataset
df_supply_bitcoin.to_csv("data/beta/bitcoin_current_supply.csv", index=False)

In [77]:
# Activamos la función y guardamos sobre la misma varible
df_supply_ethereum = clean_data(df_supply_ethereum)

In [78]:
# Guardamos en un dataset
df_supply_ethereum.to_csv("data/beta/ethereum_current_supply.csv", index=False)